In [1]:
import DSGRN
import cProfile
import sys
sys.setrecursionlimit(10**8)
sys.path.insert(0,'../src')

# please don't use from ... import *
# it makes reading code very hard for someone else
import PhenotypeGraphFun as PGF
import CondensationGraph_iter as CG
import ReducePhenotypeGraph as RPG

In [2]:
database = DSGRN.Database("../networks/ACDC_FullconnE.db") 
network = database.network

In [3]:
parameter_graph = DSGRN.ParameterGraph(network)
print(parameter_graph.size())

38416


In [4]:
AP35 = {"Hb":[0,2], "Gt":2, "Kr":0, "Kni":0}
AP37 = {"Hb":2, "Gt":[0,1], "Kr":0, "Kni":0}
AP40 = {"Hb":2, "Gt":1, "Kr":[0,1], "Kni":0} #edit
AP45 = {"Hb":[0,1], "Gt":1, "Kr":2, "Kni":0} #edit
AP47 = {"Hb":[0,1], "Gt":0, "Kr":2, "Kni":0} 
AP51 = {"Hb":1, "Gt":0, "Kr":2, "Kni":[0,1]} #edit
AP57 = {"Hb":1, "Gt":0, "Kr":[0,1], "Kni":2} #edit
AP61 = {"Hb":0, "Gt":0, "Kr":[0,1], "Kni":2}
AP63 = {"Hb":0, "Gt":[0,1], "Kr":1, "Kni":2} #edit
AP67 = {"Hb":0, "Gt":2, "Kr":1, "Kni":[0,1]} #edit

In [5]:
D = [[AP37], [AP40], [AP45], [AP47],[AP51], [AP57], [AP61], [AP63], [AP67]]
paramslist = PGF.get_paramslist_optimized(database, D, '=')

In [6]:
# cProfile.runctx('get_paramslist(database, list_of_bounds, goe)', globals(), locals())

In [7]:
edges = PGF.get_phenotype_graph_optimized(database, paramslist)

In [8]:
# cProfile.runctx('get_phenotype_graph(database, paramslist)', globals(), locals())

In [9]:
# [a,b,c] = RPG.reduce(edges, paramslist)
cg,rpl,scc = CG.condensation_graph_optimized(edges)

In [10]:
cProfile.runctx('CG.condensation_graph_optimized(edges)', globals(), locals())

         103907 function calls in 0.109 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.109    0.109 <string>:1(<module>)
        1    0.008    0.008    0.109    0.109 CondensationGraph_iter.py:101(condensation_graph_optimized)
     5271    0.000    0.000    0.000    0.000 CondensationGraph_iter.py:108(<lambda>)
        1    0.001    0.001    0.002    0.002 CondensationGraph_iter.py:109(<listcomp>)
        9    0.004    0.000    0.012    0.001 CondensationGraph_iter.py:115(<dictcomp>)
     5235    0.008    0.000    0.008    0.000 CondensationGraph_iter.py:115(<listcomp>)
        9    0.000    0.000    0.041    0.005 CondensationGraph_iter.py:116(<listcomp>)
        1    0.000    0.000    0.000    0.000 CondensationGraph_iter.py:140(<listcomp>)
       27    0.028    0.001    0.039    0.001 CondensationGraph_iter.py:4(stronglycc_iterative)
     5235    0.005    0.000    0.005    0.000 Condensa

In [11]:
print(cg)

{(0, 14): [(1, 14)], (0, 18): [(1, 18)], (1, 14): [], (1, 18): [], (2, 2655): [(3, 2403), (3, 2648)], (2, 4809): [(3, 2403), (3, 2648)], (3, 2403): [(4, 2403)], (3, 2648): [(4, 2648)], (4, 2403): [], (4, 2648): [], (5, 1): [(6, 9653), (6, 30576)], (5, 13770): [(6, 9653), (6, 30576)], (6, 9653): [(7, 9653)], (6, 30576): [(7, 30576)], (7, 9653): [], (7, 30576): [], (8, 98): [], (8, 10073): []}


In [12]:
#test case
paramslist = [[(0, 1), (0, 3), (0, 5)],[(1, 0), (1, 2), (1, 4), (1, 6), (1, 7), (1, 8), (1, 12), (1, 13)],[(2, 9), (2, 10), (2, 11)]]

edges = {(0, 1): [(1, 0), (0, 3)],(0, 3): [(1, 2), (0, 5), (0, 1), (1, 4)],(0, 5): [(0, 3), (1, 6)],(1, 0): [(1, 2), (1, 7)],(1, 2): [(1, 0)],(1, 4): [(1, 6)],(1, 6): [(1, 13), (1, 4)],(1, 7): [(1, 0), (2, 9), (1, 8)],(1, 8): [(1, 7), (2, 10)],(1, 12): [(1, 13), (2, 10)],(1, 13): [(2, 11), (1, 6), (1, 12)],(2, 9): [(2, 10)],(2, 10): [(2, 11), (2, 9)],(2, 11): [(2, 10)]}

condensation = {(2, 9): [], (1, 0): [(2, 9)], (1, 6): [(2, 9)], (0, 1): [(1, 0), (1, 6)]}

stronglycc = [[(2, 9), (2, 10), (2, 11)],[(1, 0), (1, 2), (1, 7), (1, 8)],[(1, 6), (1, 13), (1, 12), (1, 4)],[(0, 1), (0, 3), (0, 5)]]

reduced_paramslist = [[(0, 1)], [(1, 0), (1, 6)], [(2, 9)]]

In [13]:
cg,rpl,scc = CG.condensation_graph_optimized(edges)
cg == condensation
print(cg)

{(0, 1): [(1, 0), (1, 4)], (1, 0): [(2, 9)], (1, 4): [(2, 9)], (2, 9): []}


In [14]:
rpl == reduced_paramslist
print(rpl)

[[(0, 1)], [(1, 0), (1, 4)], [(2, 9)]]


In [15]:
scc == stronglycc
print(scc)

{0: [[(0, 1), (0, 3), (0, 5)]], 1: [[(1, 0), (1, 2), (1, 7), (1, 8)], [(1, 4), (1, 6), (1, 12), (1, 13)]], 2: [[(2, 9), (2, 10), (2, 11)]]}
